In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

# suppress warnings
import warnings
warnings.filterwarnings('ignore')


In [9]:
# read the data
df = pd.read_csv('https://raw.githubusercontent.com/nikjohn7/Disaster-Tweets-Kaggle/main/data/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
# how many rows and columns are in the data set?
df.shape

(7613, 5)

In [11]:
# show rows for the United States
# df[df['country'] == 'USA']

In [12]:
# show rows for Canada
# df[ df["country_name"] == 'Canada' ]

In [13]:
import nltk
nltk.download('stopwords')

stopwords = set(nltk.corpus.stopwords.words('english'))

include_stopwords = {'Crying', 'ablaze', 'Ablaze', 'fall', "calamities", 'windstorm', 'Windstorm', "fires", 'Fire', 'wounded', 'Wounded', 'sunk', 'Sunk',
                    'Survive', 'survive', 'Survived', 'survived', 'storm', 'trapped', 'tragedy'
                    }
exclude_stopwords = {'during'}

stopwords |= include_stopwords
stopwords -= exclude_stopwords
print(stopwords)

{'own', 'do', "hadn't", 'shan', 'weren', 'doesn', 'her', 'if', 'out', 'here', 'storm', 'down', 'm', 'o', 'trapped', 'mustn', 'an', 'ma', 'i', "that'll", 'some', "couldn't", 're', 'fall', 'themselves', 'such', "hasn't", 'on', "don't", 'Windstorm', 'they', 'Ablaze', 'theirs', 'wouldn', 'does', 'yours', 'further', 'now', 'over', "aren't", 'are', "mightn't", 'any', "isn't", 'below', 'to', 'but', 'few', 'most', 'Survive', 'herself', 'itself', 'same', 'his', 'Fire', 'had', 'through', 'off', 'him', 'in', 'because', "mustn't", 'is', "haven't", 'wounded', 'survived', 'ain', 'of', 'been', 'sunk', 'while', 'from', 'each', 'he', 'between', "doesn't", "you'd", 'above', 'with', 'won', 'doing', "won't", 'nor', 'ablaze', 'who', 'fires', 'under', 'tragedy', "she's", "you'll", 'will', 'yourself', "it's", 'have', 'calamities', 'has', 'about', 'haven', "you've", 've', 'no', 'mightn', 'couldn', 'all', 'them', 'himself', 'again', 'until', 'my', 'ourselves', 'Sunk', 'hadn', 'which', 'both', 'should', 'the', 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vibhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# build a text processing and classifier pipeline
# to predict the country (USA or Canada) of a speech

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

df2 = df

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df2['text'], df2['target'], test_size=0.2)

# Create a pipeline that first transforms the text data into TF-IDF vectors, then applies SVM
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', svm.SVC()),
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.77      0.89      0.82       851
           1       0.83      0.66      0.73       672

    accuracy                           0.79      1523
   macro avg       0.80      0.77      0.78      1523
weighted avg       0.79      0.79      0.78      1523



In [17]:
# This script creates a new column 'sentiment' in the dataframe, 
# which contains the sentiment score of the text. 
# The sentiment score is a float within the range [-1.0, 1.0], 
# where -1.0 denotes a very negative sentiment, 
# 1.0 denotes a very positive sentiment, 
# and values around 0 denote a neutral sentiment.

from textblob import TextBlob

# Define a function to apply sentiment analysis to a text
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # returns a value between -1 and 1

# Create a new column 'sentiment' in the DataFrame
df2['sentiment'] = df2['text'].apply(get_sentiment)

# Display the DataFrame
# df2

In [18]:
# find average sentiment for each country in df2
df2.groupby('target')['sentiment'].mean()

target
0    0.070622
1    0.018631
Name: sentiment, dtype: float64

In [19]:
# find average sentiment for each speaker in df2
# order the results from most positive to most negative

df2.groupby('keyword')['sentiment'].mean().sort_values(ascending=False).head(5)

keyword
hazardous    0.457891
razed        0.418946
outbreak     0.312661
mayhem       0.277262
wreckage     0.273440
Name: sentiment, dtype: float64

In [20]:
df2.groupby('location')['sentiment'].mean().sort_values(ascending=False)

location
The Waystone Inn               1.0
The Main                       1.0
Morocco                        1.0
Paranaque City                 1.0
Mostly Yuin.                   1.0
                              ... 
fujo garbage heaven           -1.0
Freeport IL. USA              -1.0
Mumbai india                  -1.0
sri lanka                     -1.0
Deployed in the Middle East   -1.0
Name: sentiment, Length: 3341, dtype: float64